In [1]:
!pip install -U --pre tensorflow=="2.*"

  Using cached flatbuffers-1.12-py2.py3-none-any.whl (15 kB)
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 2.0
    Uninstalling flatbuffers-2.0:
      Successfully uninstalled flatbuffers-2.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.8.0
    Uninstalling tensorflow-2.8.0:
      Successfully uninstalled tensorflow-2.8.0


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\users\\dell\\appdata\\roaming\\python\\python37\\site-packages\\~-nsorflow\\lite\\experimental\\microfrontend\\python\\ops\\_audio_microfrontend_op.so'
Consider using the `--user` option or check the permissions.



In [2]:
!pip install pycocotools

In [3]:
import os
import pathlib

if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

In [4]:
cd models/research

D:\Livestream\RealTimeObjectDetection\models\research


In [5]:
!protoc object_detection/protos/*.proto --python_out=.

In [6]:
!pip install object_detection

ERROR: Could not find a version that satisfies the requirement object_detection (from versions: none)
ERROR: No matching distribution found for object_detection


In [7]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

In [8]:
!pip install tf_slim

In [9]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [10]:
def load_model(model_name):
  base_url = 'http://download.tensorflow.org/models/object_detection/'
  model_file = model_name + '.tar.gz'
  model_dir = tf.keras.utils.get_file(
    fname=model_name, 
    origin=base_url + model_file,
    untar=True)

  model_dir = pathlib.Path(model_dir)/"saved_model"

  model = tf.saved_model.load(str(model_dir))
  model = model.signatures['serving_default']

  return model

In [11]:
PATH_TO_LABELS = 'object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [14]:
PATH_TO_TEST_IMAGES_DIR = pathlib.Path('D:/Livestream/RealTimeObjectDetection/Tensorflow/workspace/images/test')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))
TEST_IMAGE_PATHS

[WindowsPath('D:/Livestream/RealTimeObjectDetection/Tensorflow/workspace/images/test/hello.322906b0-af90-11ec-b67f-c03eba28ca1c.jpg'),
 WindowsPath('D:/Livestream/RealTimeObjectDetection/Tensorflow/workspace/images/test/hello.37013f2b-af90-11ec-97bd-c03eba28ca1c.jpg'),
 WindowsPath('D:/Livestream/RealTimeObjectDetection/Tensorflow/workspace/images/test/hello.383742dc-af90-11ec-9de6-c03eba28ca1c.jpg'),
 WindowsPath('D:/Livestream/RealTimeObjectDetection/Tensorflow/workspace/images/test/hello.811099c2-c144-11ec-ac58-c03eba28ca1c.jpg'),
 WindowsPath('D:/Livestream/RealTimeObjectDetection/Tensorflow/workspace/images/test/iloveyou.5619c7d5-af90-11ec-91bd-c03eba28ca1c.jpg'),
 WindowsPath('D:/Livestream/RealTimeObjectDetection/Tensorflow/workspace/images/test/iloveyou.5758ec04-af90-11ec-86a1-c03eba28ca1c.jpg'),
 WindowsPath('D:/Livestream/RealTimeObjectDetection/Tensorflow/workspace/images/test/iloveyou.600774be-af90-11ec-a901-c03eba28ca1c.jpg'),
 WindowsPath('D:/Livestream/RealTimeObjectDete

In [15]:
model_name = 'ssd_mobilenet_v1_coco_2017_11_17'
detection_model = load_model(model_name)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [16]:
print(detection_model.inputs)
detection_model.output_dtypes

[<tf.Tensor 'image_tensor:0' shape=(None, None, None, 3) dtype=uint8>]


{'detection_scores': tf.float32,
 'detection_classes': tf.float32,
 'detection_boxes': tf.float32,
 'num_detections': tf.float32}

In [33]:
def run_inference_for_single_image(model, image):
image = np.asarray(image)
input_tensor = tf.convert_to_tensor(image)
input_tensor = input_tensor[tf.newaxis,...]
output_dict = model(input_tensor)
num_detections = int(output_dict.pop('num_detections'))
output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
output_dict['num_detections'] = num_detectionsoutput_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
if 'detection_masks' in output_dict:detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
return output_dict

IndentationError: expected an indented block (1087299989.py, line 2)

In [28]:
def show_inference(model, image_path,class_id):
  image_np = np.array(Image.open(image_path))
  output_dict = run_inference_for_single_image(model, image_np)
  boxes = []
  classes = []
  scores = []
  for i,x in enumerate(output_dict['detection_classes']):
    if x==class_id and output_dict['detection_scores'][i] > 0.5:
      classes.append(x)
      boxes.append(output_dict['detection_boxes'][i])
      scores.append(output_dict['detection_scores'][i])
  boxes = np.array(boxes)
  classes = np.array(classes)
  scores = np.array(scores)
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      boxes,
      classes,
      scores,
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=2)
  
  display(Image.fromarray(image_np))

In [34]:
for image_path in TEST_IMAGE_PATHS:
  show_inference(detection_model, image_path, class_id)

NameError: name 'class_id' is not defined

In [30]:
for image_path in TEST_IMAGE_PATHS:
  show_inference(detection_model, image_path, 1)

TypeError: 'NoneType' object is not subscriptable